 # English-Chinese Frame Alignment (YZ)

This algorithm uses Open Multilingual WordNet to output translations of English LUs and vote for the closest Chinese frames. The results are compared against 70 manually aligned frame pairs for which F1, precision and recall are computed. 

In [18]:
import xml.etree.ElementTree as ET
import os
import re
import nltk
from hanziconv import HanziConv
from nltk.corpus import wordnet as wn
import numpy as np

# Object creation
Thanks to Steven Cen! :)

In [19]:
data_dir_str = "data/"
chin_frame_dir_str = "chin/"
eng_frame_dir_str = "bfn/"

In [20]:
class Frame_CHIN:
    def __init__(self, ID, name, name_en, dfn="---"):
        self.ID = ID
        self.name = name # TODO: changeto be named lex_name
        self.name_en = name_en
        self.dfn = dfn # no definition of frame
        
        self.lus = []
    
    def __str__(self):
        return str(self.ID) + ", " + self.name + ", (" + self.name_en + "): "# + str(self.lus[0]) + ". " # TODO: LUs

class Frame_EN:
    def __init__(self, ID, name, dfn):
        self.ID = ID
        self.name = name
        self.dfn = dfn
        
        self.lus = []
        
    def __str__(self):
        return self.name + ": " + self.dfn + ": " + (str(self.lus[0]) if len(self.lus) > 0 else "") + ".\n"

# shared between all languages
class LU:
    def __init__(self, ID, name, pos, lex_name="---", dfn="---"):
        self.ID = ID
        self.name = name
        self.pos = pos
        
        self.lex_name = lex_name # TODO: what's this
        self.dfn = dfn
        
    def __str__(self):
        return self.name

In [21]:
chin_frames = []
eng_frames = []

chin_id_to_frame = {}
eng_id_to_frame = {}

# To help find the number of frames in each language
eng_max_ID = 0
chin_max_ID = 0

In [22]:
# Parse a Chinese Frame from the XML data
def parse_chin_frame(file_name):
    global chin_max_ID
    
    tree = ET.parse(file_name)
    root = tree.getroot()

    fc = Frame_CHIN(int(root.get("id")), root.get("name"), root.get("name_en"))
    chin_max_ID = max(chin_max_ID, int(root.get("id")))
    for lu_elem in root.findall("LexicalUnit"):
        lu = LU(lu_elem.get("id"), lu_elem.get("name"), lu_elem.get("POS_MARK"))
        fc.lus.append(lu)
    
    chin_frames.append(fc)
    
    
# TODO: figure out which words don't have a certain element, although this shouldn't happen
    
def parse_eng_frame(file_name):
    global eng_max_ID
    
    tree = ET.parse(file_name)
    root = tree.getroot()
    f_eng = Frame_EN(int(root.get("ID")), root.get("name"), root.find("{http://framenet.icsi.berkeley.edu}definition").text)
    eng_max_ID = max(eng_max_ID, int(root.get("ID")))
    
    for lu_elem in root.findall("{http://framenet.icsi.berkeley.edu}lexUnit"):
        lexeme_elem = lu_elem.find("{http://framenet.icsi.berkeley.edu}lexeme")
        lu = LU(lu_elem.get("ID"), lu_elem.get("name"), lu_elem.get("POS"), lexeme_elem.get("name"), lu_elem.find("{http://framenet.icsi.berkeley.edu}definition").text)
        f_eng.lus.append(lu)
        
    eng_frames.append(f_eng)

In [23]:
# Read FrameNet Files

chin_dir = os.fsencode(data_dir_str + chin_frame_dir_str)
eng_dir = os.fsencode(data_dir_str + eng_frame_dir_str)

for file in os.listdir(chin_dir):
     file_name = os.fsdecode(file)
     if file_name.endswith(".xml"):
        parse_chin_frame(os.path.join(chin_dir.decode(), file_name))

print(chin_frames[0])


for file in os.listdir(eng_dir):
     file_name = os.fsdecode(file)
     if file_name.endswith(".xml"):
        parse_eng_frame(os.path.join(eng_dir.decode(), file_name))

print(eng_frames[1])

28, 知觉特征, (perception_description_holistic): 
Attitude_description: <def-root>The lexical units in this frame are descriptions of a <fen>Cognizer</fen>'s <fen>Attitude</fen> about or outlook on a <fen>State_of_affairs</fen>.   

<ex></ex>
<ex><fex name="Cog">Carl</fex> is becoming <fex name="Deg">more</fex> <t>pessimistic</t> <fex name="State">about the negotiations</fex> every day.</ex>

<ex>The benefits of cluelessness come from living in a society where <fex name="Cog">people</fex> <m>take</m> an <t>optimistic</t> <fex name="Att">view</fex> <fex name="State">of life</fex>.</ex>
<ex></ex>
The <fen>State_of_affairs</fen> can be expressed without a <fen>Cognizer</fen> in many cases, as shown in the following:
<ex></ex>
<ex><fex name="State">The situation</fex> is <fex name="Deg">fairly</fex> <t>grim</t> at the moment.</ex>
<ex></ex>

</def-root>: optimistic.a.



In [24]:
# Generate mapping from ID to Frame object

for chin_frame in chin_frames:
    chin_id_to_frame[chin_frame.ID] = chin_frame
for eng_frame in eng_frames:
    eng_id_to_frame[eng_frame.ID] = eng_frame

In [25]:
print(len(chin_frames), len(eng_frames))

323 1221


In [26]:
print("NON-UNIQUE")
print("Num of English LUs: " + str(sum([len(eng_frames[i].lus) for i in range(len(eng_frames))])))
print("Num of Chinese LUs: " + str(sum([len(eng_frames[i].lus) for i in range(len(chin_frames))])))
# s = 10
# len(eng_frames[s].lus)

NON-UNIQUE
Num of English LUs: 13631
Num of Chinese LUs: 3522


In [27]:
chin_lemma_to_frame = {} # chin_lemma_to_frame[LU]: maps LU to its list of frame objects 
                        # TODO: use ID's instead

# Go through each Chinese frame and map every LU to all the frames that that LU is a part of
# Note: if the same LU, ie. "in" is listed multiple times as an LU for a certain frame (ie. Conditional_occurrence) it will be counted multiple times!
for chin_frame in chin_frames:
#     print(chin_frame.name)
    for lu in chin_frame.lus:
        if lu.name in chin_lemma_to_frame:
#             if chin_frame not in chin_lemma_to_frame[lu.name]: # Only add this frame if we haven't connected this lu to chin_frame yet
            chin_lemma_to_frame[lu.name].append(chin_frame) #.append(chin_frame.name)
#                 if lu.name == "in": 
#                     print(lu.name, chin_frame.name)
        else:
            chin_lemma_to_frame[lu.name] = [chin_frame] #[chin_frame]
            
# print(chin_lemma_to_frame)
avg = sum([len(frame_list) for frame_list in chin_lemma_to_frame.values()]) / len(chin_lemma_to_frame.values())
print("Average number of Chinese frames each lemma is an LU of: " + str(avg))
print(len(chin_lemma_to_frame.values()) * avg) # sanity check. Should equal the above quantity "Num of English LUs: 3947"

Average number of Chinese frames each lemma is an LU of: 1.1935288781372846
3947.0


In [28]:
# Sample array use: weights_table[c_idx][e_idx] = "weight"
weights_table = [[0 for c in range(chin_max_ID)] for e in range(eng_max_ID)]

print (len(weights_table), len(weights_table[0])) # number of chinese frames, number of English frames

weight = {} # weight[(English frame ID, Chinese frame ID)]: weight of that alignment relationship

3020 323


# Alignment
## Find translations and add weights to table

In [29]:
for eng_frame in eng_frames:
        frame_trans_list= []
        for eng_lu in eng_frame.lus:
                stripped_eng_lu = re.sub('\s?[^\w&\s&\-].*', '', eng_lu.name)   # Remove " ()", " []", "." and POS tags to convert the English LUs to valid WordNet entries.
                #print (eng_lu)
                eng_lu_trans = []
                for i in range(len(wn.synsets(stripped_eng_lu))):
                            if wn.synsets(stripped_eng_lu)[i].lemmas(lang='cmn'):    # Use the Chinese Open Wordnet to find lemma translations
                                    syns_trans = wn.synsets(stripped_eng_lu)[i].lemmas(lang='cmn')   # A list of lemma translations of the synset using Chinese Open Wordnet.
                            else:
                                    syns_trans = wn.synsets(stripped_eng_lu)[i].lemmas(lang='qcn') # If translation can't be found in the Chinese Open Wordnet, use the Chinese Wordnet (Taiwan).
                                    #print (syns_trans)
                            # Make the translations match by POS:
                            lemma_trans = []
                            for lemma in syns_trans:
                                    if eng_lu.name.endswith('.n'):  # Nouns
                                            if '.n' in str(lemma):
                                                lemma_trans.append(re.findall(r'[\u4e00-\u9fff]+', str(lemma))[0]) # Append translated strings to list.
                                    elif eng_lu.name.endswith('.v'):  # Verbs
                                            if '.v' in str(lemma):
                                                lemma_trans.append(re.findall(r'[\u4e00-\u9fff]+', str(lemma))[0]) # Append translated strings to list.
                                    elif eng_lu.name.endswith('.a'):  # Adjectives
                                            if '.a' in str(lemma) or '.s' in str(lemma):
                                                lemma_trans.append(re.findall(r'[\u4e00-\u9fff]+', str(lemma))[0]) # Append translated strings to list.
                                    elif eng_lu.name.endswith('.adv'):  # Adverbs
                                            if '.r' in str(lemma):
                                                lemma_trans.append(re.findall(r'[\u4e00-\u9fff]+', str(lemma))[0]) # Append translated strings to list.
                                    else:  # All other cases (nums and preps)
                                            if re.search(r'[\u4e00-\u9fff]+', str(lemma)):
                                                lemma_trans.append(re.findall(r'[\u4e00-\u9fff]+', str(lemma))[0])  # Load all possible translations to list.
                            #print (lemma_trans)
                            for word in lemma_trans:
                                    word = HanziConv.toSimplified(word)    # Convert traditional Chinese characters to simplified characters
                                    eng_lu_trans.append(word)
                            eng_lu_trans = list(set(eng_lu_trans))    # Remove translation duplicates.
                #print (eng_lu_trans)
                frame_trans_list.append(eng_lu_trans)
        print (frame_trans_list)
        for lu_trans in frame_trans_list:
                for word in lu_trans:
                            if word in chin_lemma_to_frame: # If this translated version of the LU is an LU for any Chinese frame
                                    # print("\n", word, "is in", [c.name for c in chin_lemma_to_frame[word]], "\n")
                                    for chin_frame in chin_lemma_to_frame[word]:
                                            # Update weights in the table and the dictionary (TODO: more graceful way of keeping track)
                                            # TODO: merge these two to be graceful
                                            weights_table[eng_frame.ID][chin_frame.ID] += 1
                                            if (eng_frame, chin_frame) not in weight:
                                                    weight[(eng_frame, chin_frame)] = 1
                                            else:
                                                    weight[(eng_frame, chin_frame)] += 1
                 

[]
[['乐观', '乐观主义', '肯定'], [], [], ['正', '阳性', '实证主义', '带正电荷', '带正电', '实证'], ['阴性', '带负电荷', '负', '带负电'], ['无能', '不行', '糟糕'], ['阴沉', '阴郁', '沉闷', '暗', '深色', '深暗色', '黑暗', '深', '郁郁不乐', '阴暗', '隐秘', '秘密', '忧郁', '沮丧', '暗色调', '单调', '隐藏', '悲观']]
[[], []]
[['疲劳', '疲倦', '疲', '厌倦', '累'], ['困'], ['欲火中烧', '好色', '性兴奋', '性欲冲动', '欲火焚身的'], ['热望', '腹空', '渴望', '饥饿', '饿', '感到饿', '挨饿', '辘辘饥肠', '饥', '感到饥饿', '需要食物'], ['热望', '干渴', '渴望', '干燥', '口干', '干旱', '口渴', '渴'], ['欲火中烧', '角状', '好色', '性兴奋', '性欲冲动', '有角', '有角状突起', '欲火焚身的'], [], [], ['敝', '疲劳', '劳', '疲倦', '疲倦极', '空'], ['疲劳不堪', '非常疲劳', '疲倦', '疲乏不堪', '精疲力竭'], ['疲倦'], ['疲劳不堪', '非常疲劳', '疲倦', '疲乏不堪', '精疲力竭'], [], ['疲倦极', '疲劳', '疲倦'], ['疲倦极', '疲劳', '疲倦'], [], [], [], [], ['筋疲力尽'], ['倦怠', '疲劳', '乏力', '疲倦'], ['竭尽', '枯竭', '筋疲力尽', '疲惫不堪', '极度疲劳', '衰竭'], ['疲劳', '倦怠', '乏力', '疲倦'], ['疲劳', '倦怠', '乏力', '疲倦'], ['渴', '口渴'], ['充满激情', '欲火中烧', '好色', '性兴奋', '激动', '性欲冲动', '紧张', '欲火焚身的'], ['唤起', '激励', '觉醒', '睡醒'], [], ['体力缺乏', '衰弱', '虚弱', '无力'], [], [], [], ['欲吐', '令人作呕', '恶臭的', '使人

[['罢工', '击', '碰撞', '打击', '使用', '占用', '按', '击打', '碰上', '采取', '撞击', '打下来', '冲撞', '采纳', '偶然看见', '鼓动', '感染', '撞倒', '影响', '删掉', '碰击', '意外发现', '达到', '撞', '采用', '冲击', '停工', '偶然发现', '删去', '打', '得到', '感动', '击中', '使撞击'], ['轻拍', '拍击', '罢工', '停工', '攻击', '轻敲', '轻打'], []]
[['散开', '使散开', '撒', '散布', '散去', '传', '广为传播', '驱散', '分散', '铺开', '散播', '推广', '传播', '洒', '消散', '使广为人知'], ['分裂', '溶解', '化', '终止', '液化', '解散', '结束', '融化', '使溶解'], ['散开', '使散开', '撒', '伸展', '散去', '散布', '驱散', '分散', '铺开', '展开', '洒', '消散'], ['分裂', '溶解', '分解'], ['散播', '传播', '散布'], ['散播', '传播', '散布'], ['施予', '堆', '分布', '铺开', '散布', '遍散', '推广', '传阅', '使广为人知', '传', '散发', '分配', '分发', '铺', '广为传播', '给予', '分给', '散播', '传播', '使分布', '供应'], [], ['伸展', '分散', '蔓延', '铺开', '覆盖一层', '散布', '扩张', '布满', '释放', '推广', '铺盖', '放出', '使广为人知', '散开', '涂', '传', '展开', '抹', '扩散', '敷', '铺', '广为传播', '张', '散播', '传播'], [], ['洗', '降小雨', '撒', '散布', '打', '分散', '小雨', '洒'], ['广为传播', '传', '散布', '铺开', '散播', '推广', '传播', '使广为人知'], []]
[['痊愈', '复原', '恢复'], ['恢复', '康复'], ['治愈', '使恢复正常'], [

[['缓行漫步'], ['资助', '倒退', '下', '后退', '支持', '帮助', '赞成', '使倒退', '批准', '拥护', '赞助'], ['闯入', '冲入'], [], ['捆', '捆扎', '黏附', '跳动', '使黏合', '蹦跳', '系', '弹跳', '约束', '结合', '跳跃', '黏着', '粘附', '跳', '接近', '粘住', '限制', '附着', '捆绑', '联系', '束缚', '束紧', '拘束', '束', '蹦', '依附', '绑', '限定', '跃过', '粘着', '黏合', '飞跃', '套', '跳越', '跳过'], ['穿过'], ['使活跃'], ['慢跑', '使慢跑'], ['雀跃', '蹦蹦跳跳', '欢跃'], [], ['爬上', '价值增加', '攀升', '上升', '爬升', '升起', '增加', '增大', '攀登', '上涨'], [], ['畏缩', '爬', '爬泳', '匍', '自由泳', '奉承', '退缩', '匍匐', '蛇行', '挤满', '爬满', '爬行', '蠕动', '阿谀', '抖缩'], ['畏缩', '爬', '抖缩', '攀附', '匍', '长满', '奉承', '退缩', '蔓延', '蛇行', '爬行', '蠕动', '阿谀', '匍匐'], ['舞蹈', '雀跃', '舞', '跳舞', '手舞足蹈'], ['快走', '疾走', '勐冲', '冲刺'], ['款式', '风度'], ['使落空', '快走', '冲撞', '使受挫', '使破灭', '挫败', '使沮丧', '打碎', '疾走', '使碰撞', '粉碎', '勐冲', '冲刺'], [], [], ['跑来跑去'], [], ['跑来跑去'], ['加快', '疾驰', '飞驰', '赶快', '促进', '加工', '快进', '赶紧', '加紧', '冲进', '疾走', '急于', '推进', '加速'], ['加薪', '提薪'], ['提高', '远足', '增加', '徒步旅行', '勐提高', '抬高'], ['搭便车', '免费搭车'], ['妨碍', '阻碍', '束缚'], ['跃过', '跳跃', '乘飞机飞越', '轻跳', 

[['肉眼的', '裸眼的', '赤裸裸', '打', '空', '只用耳朵的'], ['打', '赤裸裸', '空'], [], ['不穿鞋', '赤脚', '赤足'], ['赤脚', '不穿鞋袜'], [], ['黯淡', '赤裸裸', '赤裸', '裸体', '凄凉的气氛'], ['赤裸', '裸体', '赤裸裸'], ['覆盖'], [], ['用坏', '表现', '经久耐用', '绑', '穿', '显出', '耐磨', '运行', '持续', '踏', '坏了', '戴', '冠', '呈现', '套', '带'], ['敷药防护', '装扮', '打扮'], ['赤脚', '不穿鞋袜'], ['朴', '刚刚充足', '无遮蔽', '赤裸裸', '抽出鞘', '仅仅', '打', '勉强', '无覆盖', '拙', '刚好够', '空', '勉强够格', '最低限度'], ['传世'], [], [], [], [], [], [], ['赤脚', '不穿鞋袜'], [], ['无顶'], [], [], ['暴露'], [], ['装扮', '打扮'], [], [], [], [], [], ['装扮', '打扮'], ['装扮', '打扮'], [], [], [], [], [], ['之间', '英寸', '间', '在职', '铟'], [], [], [], ['跑来跑去', '吹嘘'], [], []]
[['总计', '添', '产生', '加', '长', '下', '添加', '给予', '补充说', '增加', '总数', '提供', '加上', '加起来', '合计', '饶', '增添', '共计为'], ['种', '乘', '使相乘'], ['添加', '加法', '附加物', '增加', '增建物', '增加物', '增加量', '增加部分', '添加物', '附加部分', '附加'], ['增多', '倍增', '增加']]
[['添', '增长', '增加', '加大', '增大', '增'], ['增值', '增额', '增量', '增加', '加大', '增大', '增加量', '增多'], ['被占领', '落下', '倾斜', '跌落', '传下', '减轻', '被分类', '陷落', '下降', '到

[['音乐伴奏', '音乐'], ['表现', '运行', '实行', '运转', '履行', '演奏', '执行'], ['投机', '游戏', '企图获得', '戏剧', '轮', '剧本', '玩', '赌胜负', '赌博', '完全自由', '比赛回合', '容易干的事情']]
[['舔舐', '舔', '哗哗响', '舐', '发嗖嗖声', '折叠', '部分重叠', '作嗖嗖声'], [], [], [], [], ['精练', '打下', '推敲', '下', '击落'], ['啜饮', '抿', '啜', '抿吸'], [], [], [], ['宴'], ['流进', '付费给', '给小费', '餵', '流', '使满足', '付小费', '喂养', '支持', '流向', '流动', '施', '喂食', '粪', '使满意'], [], [], [], ['用餐', '进正餐', '进餐', '吃', '吃正餐', '喝'], ['吸收', '服', '被同化', '食用'], ['看护', '照顾', '挟', '护理', '照料'], [], ['伤', '食', '吃东西', '吃饭', '吃掉', '吃', '消耗'], ['敬酒', '干', '饮', '喝'], [], [], ['伤', '浪费', '挥霍无度', '挥霍', '服', '吃掉', '吃', '消耗', '食用'], [], [], [], [], ['接受', '服', '罹患', '生', '食用', '引起', '取得', '罹', '拥有', '引', '生育', '掌握', '持有', '生下来', '有', '碰', '面临', '获得', '引得', '承受', '接到', '惹', '拿', '对抗', '收到', '具有', '得到', '占有'], ['吸入', '吸收', '饮', '吸取', '喝'], [], [], [], [], ['插入', '塞入'], ['摄取', '纳入']]
[['使聚集', '见', '遭受', '适宜', '接触', '互相靠拢', '聚拢', '集会', '偶然遇见', '忍受', '遭遇', '见面', '合适', '遇见', '对', '符合', '应战', '偶然看见', '碰', '认识',

...

[['穿衣', '服装', '衣裳', '衣服', '女服', '女裙衫', '打扮', '连衣裙', '服饰', '衣着'], ['女服', '女裙衫', '连衣裙'], [], [], ['女用裙裤'], ['宽松女服', '女式内衣'], ['马靴', '马裤'], ['袜子'], [], ['护腿', '绑腿'], [], ['插条', '失礼', '滑', '条', '女式内衣', '失态', '插枝', '陶饰土', '出丑', '枕套'], ['女裙服', '女式礼服', '手术服', '礼服', '女长服', '女夜礼服'], ['菌环', '环轮', '环节', '环带', '女裙', '裙子', '裙', '孔环'], ['平角裤', '短裤', '四角裤', '短路', '衬裤', '裤衩'], ['裤子', '女式灯笼裤', '短裤', '灯笼裤'], ['裤子', '裤腿', '裤管'], [], ['衬衣', '衬衫'], ['女衬衫', '宽松短衫', '宽松的上衣'], ['遮盖物', '斗篷', '披风'], ['斗篷', '披肩'], ['涂层', '外层', '外套', '大衣', '短上衣', '外衣'], ['厚重长大衣', '厚外套'], ['厚重长大衣', '厚外套'], ['齿冠', '封套', '夹克', '外皮', '弹壳', '短上衣'], [], ['交往', '联合', '联系', '交际', '领带', '连接物', '平分', '社会交际关系', '连结', '链环', '平局', '结交'], ['袜类', '紧身裤', '男式紧身裤', '软管'], ['袜类'], ['衣服'], ['衣裳', '服饰', '服装', '衣服'], ['华丽的服饰'], ['讼案', '诉讼案', '案', '西装', '套装', '诉讼', '追求', '起诉', '控告', '官司', '求爱', '案件', '诉讼案件'], ['领带'], [], ['宽腰带'], [], [], ['蝴蝶结领带', '蝶形领结'], [], ['袜带'], [], ['女式灯笼裤', '短裤', '灯笼裤', '马裤', '裤子'], ['运动衫', '针织套衫', '毛衣'], ['跨接线', '围巾', '跳线', '工

[[], ['树皮', '三桅帆船'], [], [], [], ['爆炸', '抨击', '谴责', '轰隆声'], [], ['传声器架', '吊杆', '帆杆', '繁荣', '景气', '帆桁'], [], [], [], ['咯', '咯咯'], [], [], [], [], [], ['瓣阀'], [], ['不和', '冲突', '争执', '摩擦', '矛盾'], [], [], ['监禁', '监牢', '监狱'], [], ['咳嗽', '咳', '咳出'], [], ['缺乏', '危急情况', '困境', '艰难局面', '不足', '物资的短缺', '压碎', '粉碎'], ['无人驾驶飞机', '低音部音塞', '无人飞机', '雄蜂'], [], [], [], ['石鲈'], [], [], [], [], ['忙', '繁忙', '忙碌'], ['海鸥', '喵'], [], [], [], [], [], [], [], [], [], [], ['轻拍', '拍击', '怨', '轻敲', '轻打'], [], [], [], ['疤痕', '擦伤', '擦', '磨损'], [], [], [], [], ['味道', '海洛因', '气味', '滋味'], [], [], ['鼾声', '唿声'], [], ['呆子', '卑劣的家伙', '孬种', '蠢货', '混蛋', '不要脸的家伙', '下贱人', '无赖', '可恶的家伙', '坏蛋', '混账东西', '唿吸困难', '讨厌的人', '卑鄙的人', '恶棍', '蠢人', '兔崽子', '气绝', '可鄙的人', '狗杂种', '王八蛋', '饭桶', '愚蠢的人', '傻瓜', '笨蛋', '讨厌鬼'], [], [], ['通通'], [], ['海洛因'], [], [], ['代价', '通行费', '价值', '价格', '过桥费', '铃', '成本'], ['喇叭', '小号'], [], [], [], [], [], [], ['笛'], [], [], [], [], [], ['听觉', '音', '鸣', '声音信号'], ['随机性', '无序性', '噪音'], ['轰隆声']]
[['分遣', '分派', '派遣'], [], [

[[], ['犬吠', '吠', '汪汪叫'], [], ['吼', '咆哮'], [], ['研磨', '磨碎', '磨'], ['起潺潺声', '发汩汩声', '作汩汩声'], [], ['啼'], ['啼'], ['咯咯', '咯'], [], [], ['翘辫子', '死亡', '去世', '消逝', '死去'], [], [], ['哭泣', '大叫', '喊', '叫喊', '哭', '鸣', '悲叹', '唿喊', '尖叫', '鸣唱', '叫'], ['嗡嗡作声', '嗡嗡叫', '嗡嗡响'], ['气促', '喘息', '气喘吁吁', '喘气'], ['使发出刺耳的声音', '发出刺耳的摩擦声', '刮出刺耳声', '发刺耳声地摩擦'], ['诉冤', '呻吟'], [], ['哼'], ['作汩汩声', '发汩汩声', '潺潺而流', '起潺潺声'], ['发出嘶嘶声'], [], [], ['诉冤', '呻吟'], [], ['发出嗖嗖声', '飕飕作声', '作唿唿声', '嗡嗡转动', '发出嗡嗡声', '发飕飕声'], ['轻打', '轻敲'], [], ['喋喋不休地说', '喋喋不休'], ['大声叫喊', '吼', '咆哮'], ['低沉地说', '发隆隆声', '辘辘响', '使隆隆响'], ['唿啸', '大嚷', '大叫', '喊', '叫喊', '尖叫着说', '唿喊', '尖叫', '尖声地说', '叫', '嗷叫', '叫着说'], ['嘎嘎作响', '吱吱叫'], [], [], ['咆哮', '使缠结', '打', '纠缠', '使困惑', '使混乱', '吃', '吼叫'], ['哼着鼻子'], [], ['发噼啪声'], [], ['嘎嘎作响', '吱吱叫'], ['供认'], ['轰隆隆地移动', '轰隆地响', '发出隆隆声', '打雷', '发隆隆声'], [], [], [], [], [], [], ['喘息'], [], ['嘎嘎作响', '吱吱叫'], ['哮咳'], ['大嚷', '大叫', '喊', '叫喊', '尖叫着说', '唿喊', '尖叫', '尖声地说', '叫', '嗷叫', '叫着说'], []]
[['明明白白', '显然', '明显', '显而易见', '无疑', '显着'],

[[], ['挤', '填满', '挟', '挤向', '使卡住', '塞满', '挤入', '阻', '吃', '蜂拥进入'], ['打下', '调情的注意', '碰撞', '击', '攻', '打击', '抵达', '杀', '击打', '按', '碰上', '杀害', '撞击', '冲撞', '撞倒', '碰击', '谋杀', '达到', '撞', '残杀', '冲击', '敲', '打', '到达', '偶然碰上', '击中', '使撞击'], ['伤害', '产生剧痛', '疼', '伤', '使受伤', '刺痛', '剧痛', '疼痛', '受痛苦', '痛', '伤痛'], ['伤害', '伤', '致伤', '损害', '受伤', '使受伤', '击伤'], ['揭发', '终止', '进行', '减轻', '使戒除', '使降级', '断绝', '推翻', '陷落', '放弃', '破碎', '停止', '分离', '毁坏', '发生', '使降等', '崩落', '散去', '露出', '使放弃', '违法', '使不能成立', '中断', '使毁灭', '崩溃', '演变成', '倒塌', '透露', '坏了', '切断', '结束', '戒除', '犯罪', '违反', '散开', '倒坍', '缓冲', '用坏', '打断', '中止', '出现', '解', '解体', '使中止', '闯入', '砸', '使去掉', '破坏', '休息时间', '碎', '使中断', '暴露', '打破', '分开', '消散', '垮掉', '显现', '降低', '突然逃跑', '使降低地位', '暂停', '打', '脱离', '揭露', '泄露', '毁灭', '非法进入', '突然改变方向', '遁道', '衰减', '减少', '妨碍'], [], ['使或变的紧张', '控', '紧张起来', '勉强', '澄', '紧张'], ['折曲', '搓成', '弄弯', '捻', '拧', '弯曲', '扭动', '扭曲', '使弯曲', '捻成', '缠绕', '蠕动', '扭', '搓', '盘绕', '卷绕'], ['划船', '拉出', '划行', '行驶', '做错事', '拖', '取出', '开动', '拉', '拔出', '牵

[[], ['反复地打', '劳', '痛击', '逃税', '拍动', '逃单', '咚咚地敲', '跳动', '打成', '超过', '滴答响', '打败', '好于', '敲打', '征服', '艰难地前进', '直射', '拍打', '扑打', '扑动', '发出滴答声', '打', '赛过', '搏动', '轻敲', '胜过', '剧烈搅拌'], ['烧开', '熬', '汽化', '煮沸', '沸腾', '煮', '煮开'], ['灸', '灼', '使用', '容光焕发', '燃', '用烙器灼', '用烙器烧', '发红', '发热', '烧毁', '烧伤', '刺痛', '烧香灰', '烧光', '发光', '烧灼', '燃烧', '烧死', '灼烧', '烧', '刺'], ['强制', '胁迫', '强迫', '威胁'], ['协作', '联手', '联合', '结合'], ['挤', '痛击', '榨', '禁止', '压迫', '打败', '抑制', '压榨', '征服', '压抑'], ['剪', '缺席', '剪下', '理', '浓缩', '切', '删节', '杀', '缩短', '变淡', '精简', '割', '剪下来', '冲淡', '对付', '削', '突然转向', '削减', '裁', '做', '卸', '不闻不问', '减少', '降低', '稀释', '跳过', '略过'], ['片'], ['打'], ['打下', '调情的注意', '碰撞', '击', '攻', '打击', '抵达', '杀', '击打', '按', '碰上', '杀害', '撞击', '冲撞', '撞倒', '碰击', '谋杀', '达到', '撞', '残杀', '冲击', '敲', '打', '到达', '偶然碰上', '击中', '使撞击'], ['抱怨', '发牢骚', '悲叹', '蹴', '踹', '埋怨', '踢'], ['捅', '刺伤', '戳', '用刀切割', '用刀割伤', '用刀弄伤', '刺'], ['打洞', '穿孔于', '戳洞', '用拳勐击', '以拳痛击', '打孔', '拳打', '打眼于'], ['撞', '用力撞', '使毁灭', '推翻', '毁灭', '破坏', '打碎', '勐击', '粉碎'

## Normalization
Consider revision

In [30]:
# #Weight normalization:
# denom = [0] * len(weights_table[0])
# for eng_frame_weights in weights_table:
#         for i in range(len(eng_frame_weights)):
#                 denom[i] += eng_frame_weights[i]
# #print (denom)
# for eng_frame_weights in weights_table:
#         for i in range(len(eng_frame_weights)):
#                 if denom[i] == 0:
#                     eng_frame_weights[i] = 0
#                 else:
#                     eng_frame_weights[i] = eng_frame_weights[i]/denom[i]
# print (weights_table[5])




## Output alignments

In [31]:
f = open("eng_to_chin_best.txt","w+")

num_aligned = 0 # TODO: change name
eng__chin_alignments = {} # A dictionary that holds English and Chinese frame ID mappings.

for eng_ID in range(1, len(weights_table)):
    total_vote_count = 0
    for weight in weights_table[eng_ID]:
        total_vote_count += weight 
#     if total_vote_count == 0:
#         print (0)
#     else:
#         print (max(weights_table[eng_ID])/total_vote_count, max(weights_table[eng_ID]))
    eng_array = np.array(weights_table[eng_ID])
    best_chin_frame_ID = np.argmax(eng_array)
    if max(weights_table[eng_ID]) > 4 and max(weights_table[eng_ID])/total_vote_count > 0.09:   # Establish thresholds for winning vote count and weight distribution.
        if eng_ID in eng_id_to_frame.keys():
            s = "(" + str(eng_ID) + ") " + eng_id_to_frame[eng_ID].name + " ~= (" + str(best_chin_frame_ID) + ") " + chin_id_to_frame[best_chin_frame_ID].name
            print(s)
            f.write(s + "\n")
            num_aligned += 1
            eng__chin_alignments[str(eng_ID)] = str(best_chin_frame_ID)
    else:
        if eng_ID in eng_id_to_frame.keys():  # Skip the gaps in English frame IDs.
            s = "(" + str(eng_ID) + ") No match found for " + eng_id_to_frame[eng_ID].name
            print(s)
            f.write(s + "\n")
            eng__chin_alignments[str(eng_ID)] = '0'  # Set alignment ID to '0' is no match is found.
    
#     print(chin_ID, chin_id_to_frame[chin_ID])
#print (eng__chin_alignments)

f.close()

(5) Causation ~= (116) 因果
(6) Communication ~= (3) 陈述 
(7) Motion ~= (22) 位移
(8) No match found for Perception
(9) No match found for Reciprocality
(10) No match found for Transitive_action
(11) Duplication ~= (122) 行为重启
(12) No match found for Feigning
(13) Cause_to_amalgamate ~= (180) 结合
(15) No match found for Separating
(16) No match found for Body_movement
(17) No match found for Manipulation
(18) Posture ~= (128) 姿势变化
(19) Awareness ~= (88) 思考
(20) Becoming_aware ~= (7) 发现 
(21) Categorization ~= (8) 范畴化
(22) Cogitation ~= (88) 思考
(23) No match found for Coming_to_believe
(24) Differentiation ~= (1) 辨别
(25) Evidence ~= (118) 证明
(26) Expectation ~= (115) 预期
(27) Coming_up_with ~= (42) 发明
(28) Judgment ~= (250) 传达评价
(29) Mental_property ~= (230) 心理刺激
(31) Scrutiny ~= (283) 寻找
(32) Candidness ~= (3) 陈述 
(33) No match found for Commitment
(34) Discussion ~= (3) 陈述 
(35) Encoding ~= (3) 陈述 
(36) No match found for Gesture
(37) No match found for Hearsay
(38) Communication_manner ~= (6

(950) No match found for Law
(960) Being_located ~= (188) 位置
(962) Practice ~= (265) 练习
(964) No match found for Attributes
(965) Measurable_attributes ~= (189) 体型特征
(966) No match found for Speed_description
(967) Taking_time ~= (91) 挑选
(970) No match found for Shaped_part
(971) No match found for Architectural_part
(972) No match found for Age
(973) No match found for People_by_age
(974) No match found for People_by_origin
(978) No match found for Losing_it
(979) Grasp ~= (68) 理解
(981) No match found for Craft
(982) No match found for Idiosyncrasy
(990) No match found for Capability
(1000) No match found for Rest
(1001) No match found for Increment
(1010) No match found for Partitive
(1012) Evoking ~= (13) 记忆
(1015) No match found for Gradable_attributes
(1016) No match found for Aesthetics
(1017) No match found for Noise_makers
(1019) Religious_belief ~= (130) 宗教信仰
(1021) No match found for People_by_religion
(1023) No match found for People_by_jurisdiction
(1024) Halt ~= (105) 行为绝对

(2270) Color_qualities ~= (28) 知觉特征
(2280) No match found for Social_event_individuals
(2281) No match found for Social_event_collective
(2282) No match found for Size
(2284) No match found for Heat_potential
(2285) No match found for Mining
(2286) No match found for Subjective_temperature
(2290) No match found for Time_period_of_action
(2301) No match found for Container_focused_removing
(2302) No match found for Placing_scenario
(2303) No match found for Container_focused_placing
(2320) Cause_bodily_experience ~= (19) 身体感知
(2330) No match found for Tasting
(2340) No match found for Trying_out
(2350) No match found for Opportunity
(2360) No match found for Probability
(2370) No match found for Repayment
(2371) No match found for Bond_maturation
(2380) No match found for Popularity
(2390) No match found for Appellations
(2400) No match found for Exercising
(2411) No match found for Personal_success
(2421) No match found for Rate_description
(2425) No match found for Rate_quantification

In [32]:
#num_aligned

# Evals
## Answer key

In [33]:
# Create gold alignments to map English frame IDs to Chinese frame IDs. The current version contains 70 pairs of frames.
# "0" denotes no alignment.
gold_alignments = {'53': '90', '112': '117', '2971': '35', '69': '6', '135': '13', '1310': '38', '1341': '43', '1152': '108', '117': '210', '1534': '209', '58': '227',
                   '1370': '232', '278': '296', '2800': '307', '299': '0', '72': '120', '2724': '0', '134': '0', '22': '88', '240': '26', '1344': '26', '402': '0', '2831': '0',
                   '2591': '0', '1573': '0', '59': '234', '260': '0', '480': '0', '1348': '0', '2743': '0', '1085': '0', '2719': '0', '2860': '0', '974': '0', '2604': '0', '1615': '0',
                   '51': '0', '1000': '280', '347': '0', '2838': '0', '15': '0', '328': '0', '74': '0', '1090': '0', '2911': '0', '2131': '0', '1731': '0', '1532': '0', '505': '302',
                   '1632': '0', '2890': '0', '342': '0', '1801': '0', '241': '0', '1908': '0', '147': '0', '2350': '0', '1728': '275', '283': '0', '2668': '0', '1130': '0', '2142': '0',
                   '290': '0', '2470': '0', '235': '0', '115': '0', '352': '91', '1542': '264', '1622': '49', '2675': '0'}
print (len(gold_alignments))

70


## Precision, recall and F1

In [34]:
# Compute precision, recall and F1 score on the sample outputs.
sample_alignments = {}
for eng_frame_id in gold_alignments.keys():
    sample_alignments[eng_frame_id] = eng__chin_alignments[eng_frame_id]
#print (sample_alignments)

# Precision over ALL sample frames:
precision_count = 0
for eng_frame_id in sample_alignments.keys():
#     if sample_alignments[eng_frame_id] != '0':
#         print ("(" + eng_frame_id + ") " + str(eng_id_to_frame[int(eng_frame_id)].name) + " ~= " + "(" + sample_alignments[eng_frame_id] + ") " +
#                str(chin_id_to_frame[int(sample_alignments[eng_frame_id])].name))
#     else:
#         print (str(eng_id_to_frame[int(eng_frame_id)].name) + " < no match >")
    if sample_alignments[eng_frame_id] == gold_alignments[eng_frame_id]:
        precision_count += 1
precision = precision_count / len(sample_alignments)
print ('precision = ' + str(precision))
    
# Recall over aligned frames (''no match" cases in the gold alignments are excluded):
recall_num = 0
recall_denom = 0
for eng_frame_id in gold_alignments.keys():
    if gold_alignments[eng_frame_id] != '0':
        recall_denom += 1
        if sample_alignments[eng_frame_id] == gold_alignments[eng_frame_id]:
            recall_num += 1
recall = recall_num / recall_denom
print ('recall = ' + str(recall))

#F1 score:
f1_score = 2*(precision*recall)/(precision+recall)
print ('f1 = ' + str(f1_score))

precision = 0.7428571428571429
recall = 0.68
f1 = 0.7100401606425704
